# Examen Final

- Prender cámaras.
- Puede utilizar sus apuntes.
- Puede utilizar los notebooks del curso.
- La alumna o alumno que sea sorprendido solicitando ayuda a cualquier persona, sea ella compañera o no, será calificada con un 2.
- Cada pregunta vale un punto.
- La nota mínima es un 2.
- La nota se determinará con la siguiente fórmula $nota = \min\left(puntos + 2, 7\right)$
- Todas las funciones necesarias para resolver los ejercicios están ya importadas al notebook.

## Librerías

In [11]:
from finrisk import examen_final as ef
from scipy.interpolate import interp1d
import modules.hull_white as hw
import pandas as pd
import numpy as np
import textwrap
import random
import math

## Curva Cero Cupón

Los plazos están en días y las tasas en convención exp act/365.

In [12]:
df_curva = pd.read_excel('data/20201012_built_sofr_zero.xlsx')

In [13]:
df_curva.head()

,plazo,tasa,df
0,1,0.000811,0.999998
1,7,0.000841,0.999984
2,14,0.000780,0.999970
3,21,0.000774,0.999955
4,33,0.000781,0.999929


In [14]:
curva = interp1d(df_curva['plazo'], df_curva['tasa'], 'linear', fill_value='extrapolate')

## Cálculo Opción

Para identificar si es una Call o una Put se usa un `enum`.

In [15]:
c_p = hw.CallPut.CALL
c_p = hw.CallPut.PUT

Ver toda la documentación:

In [16]:
print(hw.zcb_call_put.__doc__)


    Calcula el valor de una call o una put sobre un bono cero cupón en el modelo de HW.
    
    params:
    
    - c_p: indica si es la opción es Call o Put. Es un `enum` de tipo CallPut. Ejemplo, c_p = CallPut.CALL.
    - strike: es el strike de la opción. Se ingresa como número. Un strike del 90% se ingresa como .9.
    - r0: es la tasa corta al momento de valorizar la opción (t = 0).
    - to: instante de tiempo en que vence la opción, expresado en años.
    - tb: instante de tiempo en que vence el bono subyacente, expresado en años. Debe ser tb > to.
    - zo: factor de descuento de mercado a tiempo t = 0 hasta to.
    - zb: factor de descuento de mercado a tiempo t = 0 hasta tb.
    - gamma: parámetro gamma del modelo HW.
    - sigma: parámetro sigma del modelo HW.
    
    return:
    
    -  el valor de la opción.
    


## Valores `gamma` y `sigma`

In [17]:
gamma = 1
sigma = .005

In [18]:
strikes = [.98, .981, .982, .983, .984, .985, .99, .991, .992, .993]
for k in strikes:
    print(k, hw.zcb_call_put(
        hw.CallPut.CALL,
        k,
        curva(.0001),
        1,
        2,
        math.exp(-curva(365)),
        math.exp(-curva(730) * 730 / 365.0),
        gamma,
        sigma
    ))

0.98 0.019553575799963863
0.981 0.0185542779311344
0.982 0.017554980062305048
0.983 0.016555682193475696
0.984 0.015556384324646233
0.985 0.014557086455816881
0.99 0.0095605971116699
0.991 0.008561299242840548
0.992 0.007562001374011196
0.993 0.006562703505181733


## Preguntas

In [19]:
try:
    q = ef.get_questions()
except Exception as e:
    print(str(e))

Sólo puedes pedir las preguntas 1 vez.


In [20]:
for qq in q:
    print(f'{qq[0]}\n')
    print(textwrap.fill(f'{qq[1]}', 80))
    print('\n')

Pregunta 1:

Explique la diferencia entre valorizar un payoff g(r(T), T) por simulación de
Montecarlo utilizando el modelo de Hull-White en la medida libre de riesgo y el
modelo de Hull-White en la medida T-forward.


Pregunta 2:

Con la curva cero cupón, interpolando linealmente en tasa, calcule la tasa
forward entre 482 y 654 días.


Pregunta 3:

Una opción sobre un bono cupón cero valorizada con el modelo de Vasicek vale lo
mismo que la misma opción valorizada con el modelo de Hull-White. Comente.


Pregunta 4:

Considere un swap a 1Y con cupones semestrales de 1.3% (lineal). Si la tasa
cupón cero a 0.5Y es 1.2% (exp), calcule la tasa cero a 1Y.


Pregunta 5:

Usando la curva cero cupón entregada y parámetros gamma = 1.0 y sigma = 0.5%
valorice una call a 1Y sobre un bono cero cupón a 2Y con un strike de 98.2%.
Entregue el resultado 8 decimales y suponiendo un nocional de 1.


Pregunta 6:

Calcule un paso de simulación de Montecarlo para el modelo de Vasicek usando
gamma = 1.0, sigm

## Respuestas

### Pregunta 1

Explique la diferencia entre valorizar un payoff g(r(T), T) por simulación de
Montecarlo utilizando el modelo de Hull-White en la medida libre de riesgo y el
modelo de Hull-White en la medida T-forward. 

El payoff que se obtiene utilizando simulaciones en la medida libre de riesgo es con una tasa fija del bono. En la medida t-forward existe un drift 
asociado a la tasa en la medida forward que es: 𝜃𝑡−𝐵(𝑡,𝑇)𝜎2−𝛾∗𝑟𝑡, esto hace que los valores del payoff sean diferentes

### Pregunta 2

In [31]:
#Con la curva cero cupón, interpolando linealmente en tasa, calcule la tasa
#forward entre 482 y 654 días.
t1=482
t2=654
r1=curva(1)
r2=curva(2)
df1=math.exp(-r1*t1/365)
df2=math.exp(-r2*t2/365)
fwd12=(df1/df2 - 1.0)*360.0 /(t2-t1)
print(f'{fwd12:.8%}')

0.08185334%


### Pregunta 3

Una opción sobre un bono cupón cero valorizada con el modelo de Vasicek vale lo
mismo que la misma opción valorizada con el modelo de Hull-White. Comente.

no dado que la tasa instantánea en la medida histórica es:
𝑑𝑟𝑡=𝛾(𝑟⎯⎯⎯−𝑟𝑡)𝑑𝑡+𝜎𝑑𝑋𝑡 y supone que   𝜆  el market price of interest rate risk es constante y por lo tanto, en la medida ajustada por riesgo:
𝑑𝑟𝑡=𝛾∗(𝑟⎯⎯⎯∗−𝑟𝑡)𝑑𝑡+𝜎𝑑𝑋∗𝑡, por otro lado el modelo hull-white es una extension del modelo vasisek en donde la medida ajustada por riesgo es: 𝑑𝑟𝑡=(𝜃𝑡−𝛾∗𝑟𝑡)𝑑𝑡+𝜎𝑑𝑋𝑡 
es decir el modelo hull white incluye La función  𝜃𝑡 que a diferencia vasisek este modelo incluye un nivel de reversion a la media variable en el tiempo, de esta manera 
  𝜃𝑡 se puede calidar para ajustarse perfectamente al valor de mercado de los bonos cupon cero. Vasisek no puede hacerlo por lo que ambas opciones darian diferente valores

### Pregunta 4

### Pregunta 5

### Pregunta 6

In [37]:
#Calcule un paso de simulación de Montecarlo para el modelo de Vasicek usando
#gamma = 1.0, sigma = 0.5%, dt = 1/264, r0 obtenido de la curva cupón cero
#entregada y número aleatorio N(0, 1) igual a .1.4756 .
gamma = 1.0
sigma = 0.005
r0=curva(0)
r_=r0
dt = 1 / (264 * 1)
dt_gamma_r_ = dt * gamma * r_
sigma_sqdt = sigma * math.sqrt(dt)
result = [(0, r0),]
gamma_dt = gamma * dt
r = r0
for i in range(1, (num_dias + 1) * pasos_dia):
    r = dt_gamma_r_ + (1 - gamma_dt) * r + sigma_sqdt * .1.4756 # Discretización de Euler
    result.append((i * dt, r))
result

SyntaxError: invalid syntax (<ipython-input-37-72bb2fe7381a>, line 15)

### Pregunta 7

In [32]:
#Calcule la TNA con ICP0 = 10,000.00 e ICP365 = 10,020.00 (365 días después).
ICP0 = 10000
ICP365 = 10020
d=365
tna=(ICP365/ICP0 -1)*d/360
print(f'tna: {tna:.4%}')

tna: 0.2028%
